In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import napari
import pandas as pd
from skimage.filters import gaussian, sobel, threshold_minimum, threshold_otsu


In [2]:
%matplotlib qt

Playing with an image analysis pipeline for analyzing axiozoom low mag images and quantifying reporter expression. Data format is a list of .tifs, with each larvae having 2 images, one for each channel. Mask by red channel and compute intensity of green channel.

In [5]:
def load_red_green_ims(image_dir, prefix, suffix, first_channel):
    ims = glob(image_dir + '/*.tif')
    assert np.mod(len(ims), 2) == 0
    num_larvae = int(len(ims) / 2)
    
    # assemble arrays for the green and red images
    for i in range(len(ims)):
        this_im_name = prefix + str("%02d" % (i,)) + suffix
        this_im = plt.imread(image_dir + '/' + this_im_name)
        this_larvae_id = int(np.floor(i / 2))
        if i == 0:
            red_ims = np.zeros((num_larvae,) + this_im.shape)
            green_ims = np.zeros((num_larvae,) + this_im.shape)
        if first_channel == 'green':
            if np.mod(i, 2) == 0:                                        # even
                green_ims[this_larvae_id] = this_im
            else:
                red_ims[this_larvae_id] = this_im
        elif first_channel == 'red':
            if np.mod(i, 2) == 0:                                        # even
                red_ims[this_larvae_id] = this_im
            else:
                green_ims[this_larvae_id] = this_im
    
    return red_ims, green_ims


def compute_mean_green_intensities(red_ims, green_ims, red_thresh, mask_red=True):
    # compute total green intensity levels
    mean_green_intensities = np.zeros(len(red_ims))
    for i in range(len(red_ims)):
        this_green = green_ims[i]
        if mask_red:
            if red_thresh is None:
                red_thresh = threshold_otsu(red_ims[i])

            red_mask = red_ims[i] > red_thresh
            this_green = red_mask * this_green
            
        these_green_intens = this_green[this_green > 0]
        mean_green_intensities[i] = np.mean(these_green_intens)
    
    return mean_green_intensities


def compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_channel, red_thresh, mask_red=True):
    red_ims, green_ims = load_red_green_ims(image_dir, prefix, suffix, first_channel)
    mean_green_intensities = compute_mean_green_intensities(red_ims, green_ims, red_thresh, mask_red)
    
    return mean_green_intensities


def compute_green_1d_dist_raw(green, red, red_thresh, green_thresh=275):
    red = gaussian(red, sigma=3)
    if red_thresh is None:
        red_thresh = threshold_otsu(red)
    red_mask = red > red_thresh
    green = green * red_mask
    green_1d = np.zeros(len(green))
    for i in range(len(green)):
        this_row = green[i]
        these_pixels = this_row[this_row > green_thresh]
        #green_1d[i] = np.mean(these_pixels)
        green_1d[i] = np.sum(these_pixels)

    
    return green_1d


def compute_mean_inten_in_3_bins(green_1d, points_df):
    bin1_intens = green_1d[int(sub_df.iloc[0].y):int(sub_df.iloc[1].y)]
    bin2_intens = green_1d[int(sub_df.iloc[1].y):int(sub_df.iloc[2].y)]
    bin3_intens = green_1d[int(sub_df.iloc[2].y):int(sub_df.iloc[3].y)]
    
    mean1 = np.nanmean(bin1_intens)
    mean2 = np.nanmean(bin2_intens)
    mean3 = np.nanmean(bin3_intens)
    
    return np.array((mean1, mean2, mean3))

def compute_mean_inten_in_fine_bins(green_1d, points_df, num_fine_bins):
    fine_bins = np.linspace(0, 1, num_fine_bins+1)

    green_1d = green_1d[int(sub_df.iloc[0].y):int(sub_df.iloc[3].y)]
    ap = np.linspace(0, 1, len(green_1d))
    these_fine_binned_intens = np.zeros((1, len(fine_bins) - 1))
    for j in range(len(fine_bins) - 1):
        these_fine_binned_intens[0, j] = np.nansum(green_1d[(ap >= fine_bins[j] ) * (ap < fine_bins[j+1])])

    return these_fine_binned_intens

def append_points_df(df1,df2):
    max_larvae_number = np.max(df1.larvae_num)
    df2.larvae_num += max_larvae_number + 1
    
    df = pd.concat((df1, df2), ignore_index=True)
    
    return df



In [126]:
# 25C 6 hr (only one with red)
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_04_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-hs-dtom_7pt5hrs'
prefix = r'exp10ms_mag23pt5_pwr100_00'
suffix = r'.tif'
first_im = 'red'
red_thresh = 400

In [128]:
# 18C 6 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_09_30_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-hs-dtom_6hrs/18C_culled'
prefix = r'18C00'
suffix = r'.tif'
first_im = 'red'
red_thresh = 400

In [130]:
# 25C 6 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_09_30_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-hs-dtom_6hrs/25C_take2'
prefix = r'l00'
suffix = r'.tif'
first_im = 'red'
red_thresh = 500

In [108]:
# mock inj
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_11_dpt-gfp_uas-mcd8-mch_r4-gal4/mock_inj_7hrs_exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'
red_thresh = 700

In [1]:
# 25C 24 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_26-dpt-gfp_uas-mcd8-mch_r4-gal4_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'
red_thresh = 500

In [131]:
# run the whole pipeline
total_green_intensities = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_channel, red_thresh, mask_red=True)
plt.figure()
plt.plot(1 * np.ones(len(total_green_intensities)), total_green_intensities, 'ko')
plt.yscale('linear')

In [23]:
plt.close('all')

In [4]:
red_thresh = 500

# total gfp intensity analysis

In [18]:
# run the pipeline for multiple datasets OLD

# 25C 6 hr (only one with red)
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_04_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-hs-dtom_7pt5hrs'
prefix = r'exp10ms_mag23pt5_pwr100_00'
suffix = r'.tif'
first_im = 'red'
#red_thresh = 400

green_0 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


# 18C 6 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_09_30_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-hs-dtom_6hrs/18C_culled'
prefix = r'18C00'
suffix = r'.tif'
first_im = 'red'
#red_thresh = 400

green_1 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


# 25C 6 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_09_30_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-hs-dtom_6hrs/25C_take2'
prefix = r'l00'
suffix = r'.tif'
first_im = 'red'
#red_thresh = 500

green_2 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


# mock inj
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_11_dpt-gfp_uas-mcd8-mch_r4-gal4/mock_inj_7hrs_exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'
#red_thresh = 700

green_3 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


# no inj
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_11_dpt-gfp_uas-mcd8-mch_r4-gal4/no_inj_7hrs_exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'
#red_thresh = 700

green_4 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


# 25C 24 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_26-dpt-gfp_uas-mcd8-mch_r4-gal4_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'
#red_thresh = 500

green_5 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)



# no inj late
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/Tiffany/10-21/2022_10_21_dpt-gfp_r4-gal4_uas-mcd8-mcherry_control/exp10ms_mag23pt5_pwr100_Second_Batch'
prefix = r'larvae00'
suffix = r'.tif'
first_im = 'red'
red_thresh = 500

green_6 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)
good_ids = [0, 1, 2, 3, 4, 6, 7, 9, 14, 15]
green_6 = green_6[good_ids]

# early 24 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_11_02_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5x'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'
green_7 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)



/home/brandon/anaconda3/envs/ome_zarr/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/brandon/anaconda3/envs/ome_zarr/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [19]:
# plot results

markersize = 8
alpha = 0.4
sigx = 0.0

plt.figure()

# # 0
# greens = green_0
# plot_id = 0
# plt.plot(plot_id *np.ones(len(greens)), greens, 'ko') 

# # 1
# greens = green_1
# plot_id = 1
# plt.plot(plot_id *np.ones(len(greens)), greens, 'ko') 

# 2
greens = np.concatenate((green_2, green_0))
plot_id = 2
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize) 

# 3
greens = green_3
plot_id = 1
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize) 

# 4
greens = np.concatenate((green_4, green_6))
plot_id = 0
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize) 

# 5
greens = green_5
plot_id = 3
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize) 

plt.yscale('log')
plt.xticks([0, 1, 2, 3], labels=['no inj.', 'saline 6hr', 'e. coli late 6hr', 'e.coli late 24hr'])
plt.ylabel('mean dpt-gfp intensity per larvae')
#plt.ylim([1.5e2, 1.5e4])
plt.ylim([180, 4000])

(180, 4000)

In [20]:
# run the pipeline for multiple datasets new

# 25C 6 hr (only one with red)
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_04_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-hs-dtom_7pt5hrs'
prefix = r'exp10ms_mag23pt5_pwr100_00'
suffix = r'.tif'
first_im = 'red'
#red_thresh = 400

green_0 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)




# 25C 6 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_09_30_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-hs-dtom_6hrs/25C_take2'
prefix = r'l00'
suffix = r'.tif'
first_im = 'red'
#red_thresh = 500

green_2 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


# mock inj late 6 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_11_dpt-gfp_uas-mcd8-mch_r4-gal4/mock_inj_7hrs_exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'
#red_thresh = 700

green_3 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


# no inj late 6 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_11_dpt-gfp_uas-mcd8-mch_r4-gal4/no_inj_7hrs_exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'
#red_thresh = 700

green_4 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


# late 24 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_26-dpt-gfp_uas-mcd8-mch_r4-gal4_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'
#red_thresh = 500

green_5 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


# no inj early 24 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/Tiffany/10-21/2022_10_21_dpt-gfp_r4-gal4_uas-mcd8-mcherry_control/exp10ms_mag23pt5_pwr100_Second_Batch'
prefix = r'larvae00'
suffix = r'.tif'
first_im = 'red'
red_thresh = 500

green_6 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)
good_ids = [0, 1, 2, 3, 4, 6, 7, 9, 14, 15]
green_6 = green_6[good_ids]

# early 24 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_11_02_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5x'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'
green_7 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


# late 24 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_11_02_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5x'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'

green_8 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)

# early 24 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/Tiffany/10-28/2022_10_28_dpt-gfp_r4-gal4_uas-mcd8-mcherry_Ecoli/exp10ms_mag23pt5_pwr100'
prefix = r'00'
suffix = r'.tif'
first_im = 'red'

green_9 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


In [32]:
# plot results

markersize = 8
alpha = 0.4
sigx = 0.0

plt.figure()


################## early ########################
facecolor = 'c'

# no inj 6 hr
plot_id = 0
greens = []
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 

# no inj 24 hr
plot_id = 1
greens = green_6
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 

# mock inj 6 hr
plot_id = 2
greens = []
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 


# mock inj 24 hr
plot_id = 3
greens = []
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 

# early 6 hr
plot_id = 4
greens = []
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 

# early 24 hr
plot_id = 5
greens = np.concatenate((green_7, green_9), axis=0)
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 


################## late #####################
facecolor = 'g'

# no inj 6 hr
greens = green_4
plot_id = 6
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 

# no inj 24 hr
plot_id = 7
greens = []
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 

# mock inj 6 hr
plot_id = 8
greens = green_3
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 

# mock inj 24 hr
plot_id = 9
greens = []
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 

# late 6 hr
plot_id = 10
greens = np.concatenate((green_2, green_0))
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 

# late 24 hr
plot_id = 11
greens = np.concatenate((green_5, green_8), axis=0)
plt.plot(plot_id *np.ones(len(greens)) + sigx * np.random.normal(size=len(greens)), greens, 'ko', alpha=alpha, markersize=markersize, markerfacecolor=facecolor) 




plt.yscale('log')
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], labels=['no inj. 6hr', 'no inj. 24 hr', 'mock 6hr', 'mock 24 hr', 'e. coli 6hr', 'e.coli 24hr', 'no inj. 6hr', 'no inj. 24 hr', 'mock 6hr', 'mock 24 hr', 'e. coli 6hr', 'e.coli 24hr'])
plt.ylabel('mean dpt-gfp intensity per larvae')
#plt.ylim([1.5e2, 1.5e4])
plt.ylim([100, 1e4])

(100, 10000.0)

In [26]:
green_4

array([131.52950139, 132.34670764, 130.38244514, 129.17672917])

# 1d dist analysis

In [9]:
"""compute 1d dists for one case and plot each larvae's dist"""
# early 24 hr
#image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/Tiffany/10-28/2022_10_28_dpt-gfp_r4-gal4_uas-mcd8-mcherry_Ecoli/exp10ms_mag23pt5_pwr100'
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_11_04_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-dtom_24hrs/early_L3/exp10ms_pwr100_mag23pt5x'

#prefix = r'00'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'

# late 24 hr
# image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_26-dpt-gfp_uas-mcd8-mch_r4-gal4_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5'
# prefix = r'im_'
# suffix = r'.tif'
# first_im = 'red'


red_thresh = None
green_thresh = 275
red_ims, green_ims = load_red_green_ims(image_dir, prefix, suffix, first_im)
green_intens = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)

df = pd.read_pickle(image_dir + '/points.pkl')

means_by_bins = np.empty((1,3))

num_fine_bins = 25
fine_bins = np.linspace(0, 1, num_fine_bins+1)
fine_binned_intens = np.empty((1, len(fine_bins) - 1))

plt.figure()
for i in range(len(green_ims)):
    sub_df = df[df.larvae_num==i]
    #if i == 1:
    #    continue

    if green_intens[i] < green_thresh:
        continue
    this_green_1d = compute_green_1d_dist_raw(green_ims[i], red_ims[i], red_thresh, green_thresh)
    these_fine_binned_intens = compute_mean_inten_in_fine_bins(this_green_1d, sub_df, num_fine_bins)
    fine_binned_intens = np.append(fine_binned_intens, these_fine_binned_intens / np.max(these_fine_binned_intens), axis=0)
    plt.plot(fine_bins[:-1], these_fine_binned_intens.T / np.max(these_fine_binned_intens))

    these_means = compute_mean_inten_in_3_bins(this_green_1d, sub_df)
    these_means = np.expand_dims(these_means, axis=1).T
    means_by_bins = np.append(means_by_bins, these_means, axis=0)
    


# plot individual late dists

In [143]:
"""late 24, combining datasets"""
red_thresh = None
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_26-dpt-gfp_uas-mcd8-mch_r4-gal4_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'

red_ims_1, green_ims_1 = load_red_green_ims(image_dir, prefix, suffix, first_im)
df_1 = pd.read_pickle(image_dir + '/points.pkl')

green_intens_1 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)

image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_11_02_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5x'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'

red_ims_2, green_ims_2 = load_red_green_ims(image_dir, prefix, suffix, first_im)
df_2 = pd.read_pickle(image_dir + '/points.pkl')

green_intens_2 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)

# red_ims = np.append(red_ims_1, red_ims_2, axis=0)
# green_ims = np.append(green_ims_1, green_ims_2, axis=0)
# df = append_points_df(df_1, df_2)
# green_intens = np.append(green_intens_1, green_intens_2, axis=0)

# red_ims = red_ims_2#np.append(red_ims_1, red_ims_2, axis=0)
# green_ims = green_ims_2#np.append(green_ims_1, green_ims_2, axis=0)
# df = df_2.copy()#append_points_df(df_1, df_2)
# green_intens = green_intens_2#np.append(green_intens_1, green_intens_2, axis=0)


red_ims = red_ims_1#np.append(red_ims_1, red_ims_2, axis=0)
green_ims = green_ims_1#np.append(green_ims_1, green_ims_2, axis=0)
df = df_1.copy()#append_points_df(df_1, df_2)
green_intens = green_intens_1#np.append(green_intens_1, green_intens_2, axis=0)


plt.figure()
for i in range(len(green_ims)):
    sub_df = df[df.larvae_num==i]
    
    if green_intens[i] < green_thresh:
        continue
    this_green_1d = compute_green_1d_dist_raw(green_ims[i], red_ims[i], red_thresh)
    these_fine_binned_intens = compute_mean_inten_in_fine_bins(this_green_1d, sub_df, num_fine_bins)
    fine_binned_intens = np.append(fine_binned_intens, these_fine_binned_intens / np.max(these_fine_binned_intens), axis=0)
    plt.plot(fine_bins[:-1], these_fine_binned_intens.T / np.max(these_fine_binned_intens))

    

# compare early and late dists

In [13]:
"""compute 1d dists and compare mean + std. error between early and late"""
# params
red_thresh = None
green_thresh = 275
num_fine_bins = 25
fine_bins = np.linspace(0, 1, num_fine_bins+1)

###################################################### early ###################################
# early 24 hr
#dateset 1
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/Tiffany/10-28/2022_10_28_dpt-gfp_r4-gal4_uas-mcd8-mcherry_Ecoli/exp10ms_mag23pt5_pwr100'
prefix = r'00'
suffix = r'.tif'
first_im = 'red'

red_ims_1, green_ims_1 = load_red_green_ims(image_dir, prefix, suffix, first_im)
green_intens_1 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


df_1 = pd.read_pickle(image_dir + '/points.pkl')

# dataset 2
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_11_04_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-dtom_24hrs/early_L3/exp10ms_pwr100_mag23pt5x'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'

red_ims_2, green_ims_2 = load_red_green_ims(image_dir, prefix, suffix, first_im)
green_intens_2 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


df_2 = pd.read_pickle(image_dir + '/points.pkl')

# combine datasets
red_ims = np.append(red_ims_1, red_ims_2, axis=0)
green_ims = np.append(green_ims_1, green_ims_2, axis=0)
df = append_points_df(df_1, df_2)
green_intens = np.append(green_intens_1, green_intens_2, axis=0)

# red_ims = red_ims_1
# green_ims = green_ims_1
# df = df_1.copy()
# green_intens = green_intens_1

fine_binned_intens = np.empty((1, num_fine_bins))
for i in range(len(green_ims)):
    sub_df = df[df.larvae_num==i]
    if i == 1:
        continue

    if green_intens[i] < green_thresh:
        continue
    this_green_1d = compute_green_1d_dist_raw(green_ims[i], red_ims[i], red_thresh, green_thresh)
    these_fine_binned_intens = compute_mean_inten_in_fine_bins(this_green_1d, sub_df, num_fine_bins)
    fine_binned_intens = np.append(fine_binned_intens, these_fine_binned_intens / np.max(these_fine_binned_intens), axis=0)
    
plt.figure()
plt.plot(fine_bins[:-1], np.nanmean(fine_binned_intens, axis=0), label='early')
lower = np.nanmean(fine_binned_intens, axis=0) - np.nanstd(fine_binned_intens, axis=0) / np.sqrt(np.sum(~np.isnan(fine_binned_intens), axis=0))
upper = np.nanmean(fine_binned_intens, axis=0) + np.nanstd(fine_binned_intens, axis=0) / np.sqrt(np.sum(~np.isnan(fine_binned_intens), axis=0))
plt.fill_between(fine_bins[:-1], lower, upper, alpha=0.25)

################################################### late #################################################
# late 24 hr
red_thresh = None

image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_26-dpt-gfp_uas-mcd8-mch_r4-gal4_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'

red_ims_1, green_ims_1 = load_red_green_ims(image_dir, prefix, suffix, first_im)
df_1 = pd.read_pickle(image_dir + '/points.pkl')

green_intens_1 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)

image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_11_02_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5x'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'

red_ims_2, green_ims_2 = load_red_green_ims(image_dir, prefix, suffix, first_im)
df_2 = pd.read_pickle(image_dir + '/points.pkl')

green_intens_2 = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)

red_ims = np.append(red_ims_1, red_ims_2, axis=0)
green_ims = np.append(green_ims_1, green_ims_2, axis=0)
df = append_points_df(df_1, df_2)
green_intens = np.append(green_intens_1, green_intens_2, axis=0)




fine_binned_intens = np.empty((1, num_fine_bins))
for i in range(len(green_ims)):
    sub_df = df[df.larvae_num==i]
    
    if green_intens[i] < green_thresh:
        continue
    this_green_1d = compute_green_1d_dist_raw(green_ims[i], red_ims[i], red_thresh, green_thresh)
    these_fine_binned_intens = compute_mean_inten_in_fine_bins(this_green_1d, sub_df, num_fine_bins)
    fine_binned_intens = np.append(fine_binned_intens, these_fine_binned_intens / np.max(these_fine_binned_intens), axis=0)
    
plt.plot(fine_bins[:-1], np.nanmean(fine_binned_intens, axis=0), label='late')
lower = np.nanmean(fine_binned_intens, axis=0) - np.nanstd(fine_binned_intens, axis=0) / np.sqrt(np.sum(~np.isnan(fine_binned_intens), axis=0))
upper = np.nanmean(fine_binned_intens, axis=0) + np.nanstd(fine_binned_intens, axis=0) / np.sqrt(np.sum(~np.isnan(fine_binned_intens), axis=0))
plt.fill_between(fine_bins[:-1], lower, upper, alpha=0.25)

# style
plt.ylim([0.4,0.9])
plt.xlim([0, 1])
plt.legend()
plt.xlabel('fraction along AP axis')
plt.ylabel('normalized intensity')


Text(0, 0.5, 'normalized intensity')

# compare mean dists to uniform expectation

In [91]:
# params
red_thresh = 275
green_thresh = 275
num_fine_bins = 25
fine_bins = np.linspace(0, 1, num_fine_bins+1)

###################################################### early ###################################
# early 24 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/Tiffany/10-28/2022_10_28_dpt-gfp_r4-gal4_uas-mcd8-mcherry_Ecoli/exp10ms_mag23pt5_pwr100'
prefix = r'00'
suffix = r'.tif'
first_im = 'red'

red_ims, green_ims = load_red_green_ims(image_dir, prefix, suffix, first_im)
green_intens = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


df = pd.read_pickle(image_dir + '/points.pkl')

fine_binned_intens = np.empty((1, num_fine_bins))
for i in range(len(green_ims)):
    sub_df = df[df.larvae_num==i]
    if i == 1:
        continue

    if green_intens[i] < green_thresh:
        continue
    this_green_1d = compute_green_1d_dist_raw(green_ims[i], red_ims[i], red_thresh)
    these_fine_binned_intens = compute_mean_inten_in_fine_bins(this_green_1d, sub_df, num_fine_bins)
    fine_binned_intens = np.append(fine_binned_intens, these_fine_binned_intens / np.max(these_fine_binned_intens), axis=0)
    
plt.figure()
plt.plot(fine_bins[:-1], np.nanmean(fine_binned_intens, axis=0), label='early')
lower = np.nanmean(fine_binned_intens, axis=0) - np.nanstd(fine_binned_intens, axis=0) / np.sqrt(np.sum(~np.isnan(fine_binned_intens), axis=0))
upper = np.nanmean(fine_binned_intens, axis=0) + np.nanstd(fine_binned_intens, axis=0) / np.sqrt(np.sum(~np.isnan(fine_binned_intens), axis=0))
plt.fill_between(fine_bins[:-1], lower, upper, alpha=0.25)


######################################################### early uniform expectation #####################

# early 24 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/Tiffany/10-28/2022_10_28_dpt-gfp_r4-gal4_uas-mcd8-mcherry_Ecoli/exp10ms_mag23pt5_pwr100'
prefix = r'00'
suffix = r'.tif'
first_im = 'red'

red_ims, green_ims = load_red_green_ims(image_dir, prefix, suffix, first_im)
green_intens = compute_mean_green_intensities_from_dir(image_dir, prefix, suffix, first_im, red_thresh, mask_red=True)


df = pd.read_pickle(image_dir + '/points.pkl')

fine_binned_intens = np.empty((1, num_fine_bins))
for i in range(len(green_ims)):
    sub_df = df[df.larvae_num==i]
    if i == 1:
        continue

    if green_intens[i] < green_thresh:
        continue
    this_red_mask = gaussian(red_ims[i], sigma=3) > red_thresh
    this_uniform_expectation = compute_green_1d_dist_raw(this_red_mask, red_ims[i], 0, 0)
    these_fine_binned_intens = compute_mean_inten_in_fine_bins(this_uniform_expectation, sub_df, num_fine_bins)
    fine_binned_intens = np.append(fine_binned_intens, these_fine_binned_intens / np.max(these_fine_binned_intens), axis=0)
    
plt.plot(fine_bins[:-1], np.nanmean(fine_binned_intens, axis=0), label='uniform')
lower = np.nanmean(fine_binned_intens, axis=0) - np.nanstd(fine_binned_intens, axis=0) / np.sqrt(np.sum(~np.isnan(fine_binned_intens), axis=0))
upper = np.nanmean(fine_binned_intens, axis=0) + np.nanstd(fine_binned_intens, axis=0) / np.sqrt(np.sum(~np.isnan(fine_binned_intens), axis=0))
plt.fill_between(fine_bins[:-1], lower, upper, alpha=0.25)


# style
plt.ylim([0.4,1.2])
plt.xlim([0, 1])
plt.legend()

In [88]:
np.sum(this_red_mask)

137697

In [58]:
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_26-dpt-gfp_uas-mcd8-mch_r4-gal4_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5'
df_1 = pd.read_pickle(image_dir + '/points.pkl')
print(df_1)
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_11_02_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5x'
df_2 = pd.read_pickle(image_dir + '/points.pkl')
print(df_2)

df = append_points_df(df_1, df_2)
print(df)

    larvae_num           y           x
0          3.0  240.143533  444.595134
1          3.0  454.276696  478.765320
2          3.0  758.391347  475.348301
3          3.0  973.663516  463.958239
4          0.0  283.425768  578.997864
5          0.0  415.550485  571.024821
6          0.0  856.345879  525.464574
7          0.0  999.860659  511.796499
8          1.0  245.838564  517.491530
9          1.0  425.801541  512.935505
10         1.0  803.951595  501.545444
11         1.0  980.497553  487.877369
12         2.0  212.807384  518.630536
13         2.0  393.909368  518.630536
14         2.0  764.086378  518.630536
15         2.0  986.192584  514.074512
16         4.0  272.035706  535.715629
17         4.0  401.882411  512.935505
18         4.0  747.001285  525.464574
19         4.0  939.493331  533.437617
20         5.0  282.286761  571.024821
21         5.0  466.805764  527.742586
22         5.0  838.121780  534.576623
23         5.0  999.860659  544.827679
    larvae_num           

In [44]:
df = append_points_df(df_1, df_2)
df

,larvae_num,y,x
0,3.0,240.143533,444.595134
1,3.0,454.276696,478.765320
2,3.0,758.391347,475.348301
3,3.0,973.663516,463.958239
4,0.0,283.425768,578.997864
5,0.0,415.550485,571.024821
6,0.0,856.345879,525.464574
7,0.0,999.860659,511.796499
8,1.0,245.838564,517.491530
9,1.0,425.801541,512.935505


In [121]:
fine_binned_intens[:,0]

array([0.        , 0.5125103 , 0.55349839, 0.20769608, 0.31774148,
       0.6160895 , 0.58691788, 0.21933138])

In [117]:
test = np.array([True, False])
~test

array([False,  True])

In [67]:
len(this_green_1d)

716

In [50]:
print(means_by_bins)
print(means_by_bins[:,0] / means_by_bins[:,1])
print(np.nanmean(means_by_bins[:,0] / means_by_bins[:,1]))
print(np.nanstd(means_by_bins[:,0] / means_by_bins[:,1]))

[[ 703.04575711  362.59448711  698.69206349]
 [ 815.12936207  598.04854767  693.1012977 ]
 [ 221.44753912  310.30636183  333.13215525]
 [ 348.91080988  289.70517389  443.09968125]
 [ 818.59979436  652.73571582  907.8306912 ]
 [ 639.35537167  629.48086546  595.38074691]
 [ 770.95329163  539.32683139  558.64475745]
 [ 989.34525778  569.8413818   314.64762672]
 [2828.5269457  2754.82094143 3518.53175146]]
[1.93893118 1.36298193 0.71364163 1.20436513 1.25410603 1.01568675
 1.42947327 1.73617657 1.02675528]
1.2980130858507293
0.3542978503719995


# create ap points

In [150]:
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_11_04_dpt-gfp_r4-gal4_uas-mcd8-mcherry_ecoli-dtom_24hrs/early_L3/exp10ms_pwr100_mag23pt5x'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'

red_ims, green_ims = load_red_green_ims(image_dir, prefix, suffix, first_im)

viewer = napari.view_image(red_ims)

/home/brandon/anaconda3/envs/ome_zarr/lib/python3.9/site-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [151]:
points = viewer.layers[1].data
print(points)
df = pd.DataFrame(points, columns=('larvae_num', 'y', 'x'))
df.to_pickle(image_dir + '/points.pkl')

[[0.00000000e+00 2.43560560e+02 5.34576623e+02]
 [0.00000000e+00 4.10994469e+02 5.39132648e+02]
 [0.00000000e+00 7.32194214e+02 5.36854635e+02]
 [0.00000000e+00 8.95072098e+02 5.43688672e+02]
 [1.00000000e+00 2.65816971e+02 6.14915061e+02]
 [1.00000000e+00 3.70993749e+02 6.18562811e+02]
 [1.00000000e+00 6.47825236e+02 6.15471590e+02]
 [1.00000000e+00 7.52394056e+02 6.19119339e+02]
 [2.00000000e+00 2.46572788e+02 5.09078895e+02]
 [2.00000000e+00 4.03426018e+02 5.21238060e+02]
 [2.00000000e+00 8.45720254e+02 5.66730911e+02]
 [2.00000000e+00 1.01024588e+03 5.62594641e+02]]


In [34]:
sub_df = df[df.larvae_num==10]
bin1_intens = this_green_1d[int(sub_df.iloc[0].y):int(sub_df.iloc[1].y)]
bin1_intens


array([3655.67065868, 3648.7202381 , 3668.82634731, 3693.1626506 ,
       3702.46666667, 3680.14457831, 3644.0239521 , 3630.94610778,
       3660.33333333, 3648.8597561 , 3627.04878049, 3606.83536585,
       3586.27607362, 3577.55214724, 3562.91411043, 3562.84662577,
       3521.3597561 , 3484.79878049, 3438.96363636, 3423.61212121,
       3411.62424242, 3388.01212121, 3338.26060606, 3249.74251497,
       3192.22155689, 3126.00598802, 3037.18562874, 2953.18562874,
       2870.81437126, 2820.46706587, 2763.56547619, 2738.28571429,
       2709.5297619 , 2692.24404762, 2677.73964497, 2659.4       ,
       2669.52352941, 2702.55294118, 2730.25146199, 2789.0994152 ,
       2835.44444444, 2865.76023392, 2872.29651163, 2920.53488372,
       2954.20348837, 2983.56647399, 3026.37572254, 3093.04624277,
       3126.1091954 , 3197.44508671, 3243.91954023, 3270.33142857,
       3286.84      , 3321.68571429, 3359.50857143, 3384.52272727,
       3403.31073446, 3425.71186441, 3451.10169492, 3441.07865

In [109]:
# check red seg
# early 24 hr
# image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/Tiffany/10-28/2022_10_28_dpt-gfp_r4-gal4_uas-mcd8-mcherry_Ecoli/exp10ms_mag23pt5_pwr100'
# prefix = r'00'
# suffix = r'.tif'
# first_im = 'red'

# late 24 hr
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_26-dpt-gfp_uas-mcd8-mch_r4-gal4_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'

red_ims, green_ims = load_red_green_ims(image_dir, prefix, suffix, first_im)

# red_ims_masked = np.zeros_like(red_ims)
# red_thresh = 500
# for i in range(len(red_ims)):
#     red_mask = red_ims[i] > red_thresh
#     red_ims_masked[i] = red_mask * red_ims[i]
# red_blurred = np.zeros_like(red_ims)
# for i in range(len(red_blurred)):
#     red_blurred[i] = gaussian(red_ims[i], sigma=3)
viewer = napari.view_image(green_ims)

red_norm = np.zeros_like(red_ims)
for i in range(len(red_norm)):
    #this_red = gaussian(red_ims[i], sigma=3)
    this_red = red_ims[i]

    red_norm[i] = (this_red - np.min(this_red)) / (np.max(this_red)- np.min(this_red))
viewer = napari.view_image(red_norm)

In [98]:
viewer.add_image(red_ims)

<Image layer 'red_ims' at 0x7f9d62196280>

In [100]:
viewer = napari.view_image(sobel(gaussian(red_ims[0], sigma=2)))

In [107]:
plt.figure()
plt.hist(np.ndarray.flatten(green_ims[0]))

(array([1.296227e+06, 1.593900e+04, 1.700900e+04, 2.347400e+04,
        3.199100e+04, 3.036900e+04, 1.686800e+04, 6.503000e+03,
        1.522000e+03, 9.800000e+01]),
 array([  99.,  740., 1381., 2022., 2663., 3304., 3945., 4586., 5227.,
        5868., 6509.]),
 <BarContainer object of 10 artists>)

In [128]:
red = red_ims[4]
red = gaussian(red, sigma=3)
red_mask = red > red_thresh

red_masked = red_mask * red
red_masked = red_masked[red_masked > 0]
plt.figure()
plt.hist(np.log10(red_masked), bins=100)

(array([1590., 1358., 1285., 1158., 1048.,  933.,  991.,  774.,  746.,
         683.,  612.,  491.,  507.,  479.,  482.,  478.,  499.,  614.,
         716.,  690.,  577.,  642., 1080., 1158., 1097., 1277., 1488.,
        1485., 1802., 1751., 1810., 2036., 1988., 2119., 2086., 1898.,
        1983., 2107., 2309., 2474., 2756., 2920., 3206., 3389., 3809.,
        3462., 2876., 2479., 2208., 1937., 1806., 1701., 1709., 1562.,
        1445., 1225., 1119., 1090., 1078.,  995.,  863.,  813.,  687.,
         589.,  520.,  471.,  386.,  332.,  247.,  310.,  353.,  310.,
         370.,  371.,  306.,  276.,  292.,  216.,  142.,  124.,  141.,
         113.,  121.,  124.,  108.,  127.,   66.,   75.,   46.,   49.,
          52.,   44.,   42.,   41.,   35.,   27.,   30.,   20.,   17.,
          17.]),
 array([2.4393337 , 2.44668716, 2.45404063, 2.4613941 , 2.46874757,
        2.47610104, 2.4834545 , 2.49080797, 2.49816144, 2.50551491,
        2.51286838, 2.52022185, 2.52757531, 2.53492878, 2.54228225

In [116]:
bins = np.logspace(2.1, 3.2, 100+1)
bin_widths = np.diff(bins)
counts, _bins = np.histogram(red_masked, bins)
probdens = counts / np.sum(counts) / bin_widths
plt.figure()
plt.plot(bins[:-1], probdens)
plt.xscale('log')
plt.yscale('log')
plt.xlim([100,5000])

(100, 5000)

In [123]:
10**2.9

794.3282347242813

In [129]:
thresh = threshold_minimum(red)
print(thresh)

366.416512458406


In [126]:
np.log10(475.26)

2.6769312636734695

In [130]:
366 / np.max(red)

0.2447938927341871

In [17]:
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/Tiffany/10-28/2022_10_28_dpt-gfp_r4-gal4_uas-mcd8-mcherry_Ecoli/exp10ms_mag23pt5_pwr100'
prefix = r'00'
suffix = r'.tif'
first_im = 'red'

red_ims, green_ims = load_red_green_ims(image_dir, prefix, suffix, first_im)

red_thresh = np.zeros_like(red_ims)
for i in range(len(red_ims)):
    thresh = threshold_minimum(red_ims[i])
    thresh = threshold_otsu(red_ims[i])
    red_thresh[i] = red_ims[i] > thresh

viewer = napari.view_image(red_thresh)

In [16]:
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/2022_10_26-dpt-gfp_uas-mcd8-mch_r4-gal4_ecoli-dtom_24hrs/exp10ms_pwr100_mag23pt5'
prefix = r'im_'
suffix = r'.tif'
first_im = 'red'


viewer = napari.view_image(red_ims)
viewer.add_image(green_ims)

<Image layer 'green_ims' at 0x7faa62925e80>

In [18]:
image_dir = r'/media/brandon/Data1/Brandon/fly_immune/bb8/Tiffany/10-28/2022_10_28_dpt-gfp_r4-gal4_uas-mcd8-mcherry_Ecoli/exp10ms_mag23pt5_pwr100'
prefix = r'00'
suffix = r'.tif'
first_im = 'red'


viewer = napari.view_image(red_ims)
viewer.add_image(green_ims)

<Image layer 'green_ims' at 0x7faa61921df0>